## Plug the File name here

In [23]:
#################################
#**  Change file name below  **

testfile_name = "data/testDatasetExample.xls"

#*********************************

test_df = pd.read_excel(testfile_name)


### Import all necessary packages

In [24]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_classif

from scipy import stats as st
from sklearn.preprocessing import PowerTransformer

import pickle

### Loading all the model objects

In [25]:
# load the imputer from disk
filename = 'saved_model/imputer.sav'
si = pickle.load(open(filename, 'rb'))

# load the transformer from disk
filename = 'saved_model/trnsformer.sav'
transformer = pickle.load(open(filename, 'rb'))


# load the scaler from disk
filename = 'saved_model/scaler.sav'
scaler = pickle.load(open(filename, 'rb'))

# Continuous columns list
float_cols = pickle.load(open('saved_model/continuous_features.pkl', 'rb'))['continuous_features']

# Categorical columns
cat_cols = pickle.load(open('saved_model/categorical_features.pkl', 'rb'))['categorical_features']

## Loadin the model
model = tf.keras.models.load_model('saved_model/best_model_aucc82')


# Preprocessing data

**Making the data frame from the  Columns used in the training**

In [26]:
df_cats = test_df[cat_cols]
df_floats = test_df[float_cols]
X = pd.concat([df_cats.reset_index(drop=True),df_floats.reset_index(drop=True)],axis=1)

### Filling missing values

In [27]:
X = pd.DataFrame(si.transform(X),columns=X.columns)

### Making features normal

In [28]:
X = pd.DataFrame(transformer.transform(X),columns=X.columns)

### Scalling the data

In [29]:
X = pd.DataFrame(scaler.transform(X),columns=X.columns)

### Predict the values

In [30]:
predictions = model.predict(X)

1/1 [==============================] - 5s 5s/step


### Fomatting the predictions

In [44]:
preds_df = pd.DataFrame(zip(test_df["ID"].to_list(),
                            list(np.squeeze(predictions))),
                            columns=(["ID","PCR"]))

### Writing result to csv

In [45]:
preds_df.to_csv('pcr_predictions.csv',index=False)